## Rozpoznawanie samochodów w czasie rzeczywistym z użyciem sieci neuronowych

---

W tym etapie użyje częściowo gotowego zbioru danych i użyje go do trenowania modelu.

Użyty zbiór danych: https://www.kaggle.com/datasets/sshikamaru/car-object-detection/data

Użyta architektura : https://github.com/ultralytics/yolov5

---

### Zacznę od wczytania potrzebnych bibliotek

In [ ]:
import torch
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import os
!pip install ultralytics
from ultralytics import YOLO
import PIL
from google.colab import userdata


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 949.8/949.8 kB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 79.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

Klonuje repozytorium yolov5

In [ ]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 17279, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 17279 (delta 3), reused 0 (delta 0), pack-reused 17270 (from 3)
Receiving objects: 100% (17279/17279), 16.14 MiB | 15.81 MiB/s, done.
Resolving deltas: 100% (11861/11861), done.


Pobieram z kaggla zbiór danych

In [ ]:
os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')
os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')

In [ ]:
!kaggle datasets download -d sshikamaru/car-object-detection
!unzip car-object-detection

Dataset URL: https://www.kaggle.com/datasets/sshikamaru/car-object-detection
License(s): other
car-object-detection.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  car-object-detection.zip
  inflating: data/sample_submission.csv  
  inflating: data/testing_images/vid_5_25100.jpg  
  inflating: data/testing_images/vid_5_25120.jpg  
  inflating: data/testing_images/vid_5_25140.jpg  
  inflating: data/testing_images/vid_5_25160.jpg  
  inflating: data/testing_images/vid_5_25180.jpg  
  inflating: data/testing_images/vid_5_25200.jpg  
  inflating: data/testing_images/vid_5_25220.jpg  
  inflating: data/testing_images/vid_5_25240.jpg  
  inflating: data/testing_images/vid_5_25260.jpg  
  inflating: data/testing_images/vid_5_26320.jpg  
  inflating: data/testing_images/vid_5_26400.jpg  
  inflating: data/testing_images/vid_5_26420.jpg  
  inflating: data/testing_images/vid_5_26560.jpg  
  inflating: data/testing_images/vid_5_26580.jpg  
  infl

Chce pobrany zbiór danych doprowadzić do uporządowanej struktury

In [ ]:
try:
  os.rename('data', 'samochody')
except FileNotFoundError:
  print("nazwa katalogu juz zmieniona")

os.mkdir('samochody/images')


In [ ]:
try:
  os.rename('samochody/training_images', 'samochody/train')
  os.rename('samochody/testing_images', 'samochody/test')
except FileNotFoundError:
  print("nazwy katalogów juz zmienione")

import shutil

try:
  shutil.move('samochody/train', 'samochody/images/')
  shutil.move('samochody/test', 'samochody/images/')
except FileNotFoundError:
  print("katalogi juz przeniesione")


Następnie musze przygotować zbiór danych w odpowiednim formacie do trenowania.

W tym przypadku bounding boxy dla danych uczących są gotowe, jedynie należy przeprowadzić transformację do odpowiedniego formatu.

In [ ]:
csv_path = "samochody/train_solution_bounding_boxes (1).csv"
df = pd.read_csv(csv_path)
df.head()

,image,xmin,ymin,xmax,ymax
0,vid_4_1000.jpg,281.259045,187.035071,327.727931,223.225547
1,vid_4_10000.jpg,15.163531,187.035071,120.329957,236.430180
2,vid_4_10040.jpg,239.192475,176.764801,361.968162,236.430180
3,vid_4_10020.jpg,496.483358,172.363256,630.020260,231.539575
4,vid_4_10060.jpg,16.630970,186.546010,132.558611,238.386422


Dla danych uczących:

In [ ]:
os.makedirs('samochody/labels/train')
for _, row in df.iterrows():
    it = "/images/train/"
    lt = "/labels/train/"
    image_path = "samochody" + it + row['image']
    xmin, ymin, xmax, ymax = row[['xmin', 'ymin', 'xmax', 'ymax']]

    img = PIL.Image.open(image_path)
    img_width, img_height = img.size

    x_center = ((xmin + xmax) / 2) / img_width
    y_center = ((ymin + ymax) / 2) / img_height
    width = (xmax - xmin) / img_width
    height = (ymax - ymin) / img_height

    txt_path = "samochody" + lt + row['image'].split('.')[0] + ".txt"
    with open(txt_path, 'a') as f:
        f.write(f"{0} {x_center} {y_center} {width} {height}\n")

Dla danych testowych bounding boxy stworzyłem używając label-studio, ponieważ nie ma ich w oryginalnym zbiorze. Etykiety umieściłem w moim repozytroium

In [ ]:
!git clone https://github.com/iochgsthsu/rozpoznawanie/

Cloning into 'rozpoznawanie'...
remote: Enumerating objects: 489, done.
remote: Counting objects: 100% (489/489), done.
remote: Compressing objects: 100% (480/480), done.
remote: Total 489 (delta 3), reused 486 (delta 3), pack-reused 0 (from 0)
Receiving objects: 100% (489/489), 3.06 MiB | 22.08 MiB/s, done.
Resolving deltas: 100% (3/3), done.


In [ ]:
try:
  shutil.move('rozpoznawanie/samochody/labels/test', 'samochody/labels/')
except FileNotFoundError:
  print("katalogi juz przeniesione")

In [ ]:
os.mkdir('datasets')
shutil.move('samochody', 'datasets/')
shutil.move('rozpoznawanie/dataset.yaml', 'datasets/')

'datasets/dataset.yaml'

Następnie tworze plik konfiguracyjny zbioru danych

In [ ]:
try:
  c = open('datasets/dataset.yaml', "r", encoding="utf-8")
  print(c.read())
  c.close()
except FileNotFoundError:
  print("Nie znaleziono pliku")

# moj_dataset.yaml
path: samochody/
train: images/train/
val: images/test/
nc: 1  
names: ['samochód']  # Nazwy klas


Na początku wczytam nowy, niewytrenowany model z architekturą YOLOv5s

In [ ]:
try:
  y = open('yolov5/models/yolov5s.yaml', "r", encoding="utf-8")
  print(y.read())
  y.close()
except FileNotFoundError:
  print("Plik nie został znaleziony")
model = YOLO("yolov5s.yaml")

# Ultralytics 🚀 AGPL-3.0 License - https://ultralytics.com/license

# Parameters
nc: 80 # number of classes
depth_multiple: 0.33 # model depth multiple
width_multiple: 0.50 # layer channel multiple
anchors:
  - [10, 13, 16, 30, 33, 23] # P3/8
  - [30, 61, 62, 45, 59, 119] # P4/16
  - [116, 90, 156, 198, 373, 326] # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [
    [-1, 1, Conv, [64, 6, 2, 2]], # 0-P1/2
    [-1, 1, Conv, [128, 3, 2]], # 1-P2/4
    [-1, 3, C3, [128]],
    [-1, 1, Conv, [256, 3, 2]], # 3-P3/8
    [-1, 6, C3, [256]],
    [-1, 1, Conv, [512, 3, 2]], # 5-P4/16
    [-1, 9, C3, [512]],
    [-1, 1, Conv, [1024, 3, 2]], # 7-P5/32
    [-1, 3, C3, [1024]],
    [-1, 1, SPPF, [1024, 5]], # 9
  ]

# YOLOv5 v6.0 head
head: [
    [-1, 1, Conv, [512, 1, 1]],
    [-1, 1, nn.Upsample, [None, 2, "nearest"]],
    [[-1, 6], 1, Concat, [1]], # cat backbone P4
    [-1, 3, C3, [512, False]], # 13

    [-1, 1, Conv, [256, 1, 1]],
    [-1, 1, nn.Upsample, [None, 2, 

Następnie przechodze do trenowania modelu

In [ ]:
results = model.train(data="datasets/dataset.yaml", epochs=10, batch=16)

Zapisuje model

In [ ]:
model.save("yolo_moj_model.pt")

Dalszy trening będe wykonywał za pomocy skryptu trening.py

In [ ]:
try:
  t = open('trening.py', "r", encoding="utf-8")
  print(t.read())
  t.close()
except FileNotFoundError:
  print("Pilk nie został znaleziony")

Pilk nie został znaleziony


Zipuje katalog runs

In [ ]:
folder_to_zip = '/content/runs'

output_zip_file = '/content/runs.zip'

try:
  shutil.make_archive(output_zip_file.replace('.zip', ''), 'zip', folder_to_zip)
except FileNotFoundError:
  print("Plik nie został znaleziony")


print(f"Folder {folder_to_zip} został spakowany do {output_zip_file}")

Folder /content/runs został spakowany do /content/runs.zip


---